In [9]:
import requests
from bs4 import BeautifulSoup as bs
import json
import pandas as pd
from rich.progress import Progress
from requests.adapters import HTTPAdapter
import xlsxwriter

headers = {
     "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/125.0.0.0 Safari/537.36"
}

In [10]:
bookGroupIDList="https://www.ptpress.com.cn/hotBook/getParentTagIdList"
groupID=[]

groupIdRaw=requests.get(bookGroupIDList,headers=headers)
groupIdList=json.loads(groupIdRaw.content)
groupIdListData=groupIdList["data"]
#print(groupIdListData)
for eachId in groupIdListData:
    parentTagID=eachId["PARENTTAGID"]
    parentTagName=eachId["PARENTTAGNAME"]
    groupID.append([parentTagID,parentTagName])
#print(groupID)

In [11]:
getBookListUrl="https://www.ptpress.com.cn/hotBook/getHotBookList?parentTagId="
getBookUrl="https://www.ptpress.com.cn/bookinfo/getBookDetailsById"
allBooksCollection=[]
allTotal=0
for eachID in groupID:
    with Progress() as progress:
        status=progress.add_task("Running",total=10000)
        curId=eachID[0]
        trialGet=requests.get(getBookListUrl+curId+"&rows=1&page=1",headers=headers)
        #print(trialGet.content)
        trialGetJson=json.loads(trialGet.content)
        totalOfGroup=trialGetJson["data"]["total"] # to get total number of books in this category
        allTotal+=totalOfGroup
        print(allTotal)
        rows=50
        curPage=1
        while(rows*(curPage-1)<totalOfGroup):
            progress.update(status,advance=50/allTotal*10000)
            reqSess=requests.session()
            reqSess.mount('https://',HTTPAdapter(max_retries=10))
            getBooks=reqSess.get(getBookListUrl+curId+"&rows="+str(rows)+"&page="+str(curPage),headers=headers)
            getBooksJson=json.loads(getBooks.content)
            books=getBooksJson["data"]["rows"]
            for eachBook in books:
                bookID=eachBook["bookId"]
                #print(getBookUrl+bookID)
                bookDetailRaw=reqSess.post(getBookUrl,headers=headers,data={"bookId":bookID},timeout=60)
                bookDetailJson=json.loads(bookDetailRaw.content)
                #print(bookDetailJson)

                jsonDatParams1=["bookName","author","bookTypeName","discountPrice","bookDiscount","price"]
                jsonDatParams2=["pages","isbn","publishDate"]
                jsonDatParams3=["data"]
                
                singleBookDetail=[]

                for param in jsonDatParams1:
                    try:
                        singleBookDetail.append(bookDetailJson["data"][param])  
                    except KeyError as e:
                        print(e)
                        print("Missing json info,ignore.")
                        singleBookDetail.append("NULL")
                for param in jsonDatParams2:
                    try:
                        singleBookDetail.append(bookDetailJson["data"]["bookDetail"]["data"][param])
                    except KeyError as e:
                        print(e)
                        print("Missing json info,ignore.")
                        singleBookDetail.append("NULL")
                for param in jsonDatParams3:
                    try:
                        singleBookDetail.append(bookDetailJson["data"]["resume"][param])
                    except KeyError as e:
                        print(e)
                        print("Missing json info,ignore.")
                        singleBookDetail.append("NULL")

                allBooksCollection.append(singleBookDetail)
                    #print([bookName,bookAuthor,bookType,bookSummary,bookDiscPrice,bookPrice,bookPage,bookISBN,bookPubDate])
            curPage+=1

Output()

Output()

Output()

Output()

Output()

...

Output()

Output()

Output()

Output()

35474


In [12]:
finalData=pd.DataFrame(allBooksCollection)
workbook=xlsxwriter.Workbook('books.xlsx')
header=["bookName","bookAuthor","bookType","bookDisPrice","bookDisPerc","bookOrgPrice","bookPages","bookISBN","bookDate","bookIntro"]
finalData.to_excel("books.xlsx",header=header,index=False)

ValueError: Writing 0 cols but got 10 aliases